Load the tips dataset.

Create a column named price_per_person. This should be the total bill divided by the party size.

Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

Use select k best to select the top 2 features for predicting tip amount. What are they?

Use recursive feature elimination to select the top 2 features for tip amount. What are they?

Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pydataset import data
import wrangle
import prepare
import math

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = data ('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df['tip_percentage'] = round((df['tip'] / df['total_bill'])*100 , 2)

In [6]:
df ['price_per_person']=  df['total_bill'] / df['size']

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,5.94,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,16.05,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,16.66,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,13.98,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,14.68,6.147500


In [8]:
#total_bill and size would be most important features for predicting the tip amount
#split data in train, validate and split
train, validate, test = wrangle.split_data(df)

train -> (136, 9)
validate -> (59, 9)
test -> (49, 9)


In [9]:
#split the target and the features
X_train = train.drop(columns = ['tip'])
y_train = train['tip']

In [10]:
X_validate = validate.drop(columns = ['tip'])
X_test = test.drop(columns = ['tip'])

In [11]:
#get all numerics columns
cols = X_train.select_dtypes(exclude='object').columns.to_list()

In [12]:
#scaled the columns
X_train_scaled , X_validate_scaled , X_test_scaled = prepare.scaled_mimmax(cols, X_train , X_validate, X_test)

In [13]:
X_train_scaled.head()

,total_bill,size,tip_percentage,price_per_person
19,0.307114,0.4,0.252853,0.150344
173,0.092355,0.2,1.000000,0.032258
119,0.206805,0.2,0.161850,0.182796
29,0.411622,0.2,0.240996,0.452194
238,0.657534,0.2,0.000000,0.775647


In [14]:
f_selector = SelectKBest(score_func=f_regression, k=2)
f_selector.fit(X_train_scaled, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7fc3642b6160>)

In [15]:
#get the top 2 features
mask = f_selector.get_support()
X_train_scaled.columns[mask]

Index(['total_bill', 'size'], dtype='object')

In [16]:
lm = LinearRegression()
rfe = RFE(estimator=lm, n_features_to_select=2)
rfe.fit(X_train_scaled, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [17]:
rfe.support_

array([ True, False,  True, False])

In [18]:
#get the top 2 features

X_train_scaled.columns[rfe.support_]

Index(['total_bill', 'tip_percentage'], dtype='object')

In [19]:
pd.Series(dict(zip(X_train_scaled.columns, rfe.ranking_))).sort_values()

total_bill          1
tip_percentage      1
size                2
price_per_person    3
dtype: int64

In [21]:
#the top 2 features for SelectKBest are: total_bill', 'size'
#the top 2 features for Recursive Feature Elimination(RFE) are: 'total_bill', 'tip_percentage'